<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_server_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#질답세션 리턴 코드 import해서 서버에서 구동하는 코드(서버)

In [ ]:
!pip install faiss-cpu #서버 업로드 시 삭제
!pip install -U sentence-transformers #서버 업로드 시 삭제
!pip install datasets

import numpy as np
import os
import pandas as pd
import urllib.request
import faiss
import time
import json
import ast
import torch
from flask import Flask, request, jsonify
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AutoModel, AutoTokenizer
from datasets import Dataset, load_dataset
from sentence_transformers import SentenceTransformer

In [ ]:
from google.colab import drive
#구글 드라이브 마운트
drive.mount('/content/drive')

In [ ]:
#캐싱 필요. 서버에 데이터 캐싱을 해야 함.
#csv파일(혹은 데이터프레임 형태로 캐싱?), feature extraction 모델, FAISS 인덱싱 이 셋을 캐싱해둬야 함.

In [ ]:
app = Flask(__name__)

In [ ]:
#모델 경로를 글로벌 변수로 지정
global HUGGINGFACE_MODEL_PATH = 'BM-K/KoSimCSE-roberta-multitask'

In [ ]:
#검색 함수에서 사용할 사용자 입력 인코딩 함수
def encode(query):
  inputs = tokenizer(query, padding = True, truncation = True, return_tensors = 'pt')

  with torch.no_grad():
        outputs = model(**inputs)

  #배치 단위가 아니라 단일 문장이기 때문에 mean pooling을 할 때 dim = 0으로 설정
  mean_pooling = torch.mean(outputs.last_hidden_state, dim = 0)
  return mean_pooling

In [ ]:
#검색 함수
def search(query):
  t = time.time()
  query_vector = encode(query)
  k = 20
  top_k = index.search(query_vector, k)
  print('total time: {}'.format(time.time() - t))

  return [{'question': data.iloc[_id]['question'], 'answer': data.iloc[_id]['answer'], 'question_summary': data.iloc[_id]['question_summary'], 'answer_summary': data.iloc[_id]['answer_summary']} for _id in top_k[1].tolist()[0]]

In [ ]:
#초기 호출 시 모델과 토크나이저를 전역 변수로 서버에 캐싱하는 함수
def load_model_tokenizer():
    model = AutoModel.from_pretrained(HUGGINGFACE_MODEL_PATH)
    tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
    return model, tokenizer

In [ ]:
#초기 호출 시 csv 데이터를 데이터프레임으로 변형, 데이터프레임과 FAISS 인덱스를 전역 변수로 서버에 캐싱하는 함수
def load_df_index():
    #데이터 불러오기
    test_file_path = '/content/drive/My Drive/sentence_embed_result.csv'
    df = pd.read_csv(test_file_path)

    #csv파일로 저장할 때 쉼표를 살리기 위해 json.dumps로 저장했기 때문에 json.loads 사용
    df['embedding'] = df['embedding'].apply(json.loads)
    encoded_data = torch.tensor(df['embedding'].tolist())

    #속도를 위해 임베딩 열을 제외한 데이터프레임을 설정
    #인덱싱 속도를 위해서 데이터프레임을 리스트로 변환
    data = df.drop(['embedding'], axis = 1)

    # FAISS 인덱스 구축
    index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
    index.add_with_ids(encoded_data, np.array(range(0, len(data))))
    faiss.write_index(index, 'question-answer')
    return df, index

In [ ]:
#첫 실행에 호출되며 데이터를 캐싱하는 함수
model = None
tokenizer = None
index = None
df = None

@app.before_first_request
def load_resources():
    global model, tokenizer, index, df
    if model is None or tokenizer is None or index is None or df is None:
        model, tokenizer = load_model_tokenizer() # 모델, 토크나이저 로딩 함수
        df, index = load_df_index() #데이터프레임, 인덱스 로딩 함수

In [ ]:
@app.route('/search', methods=['GET'])
def find_similar():
    user_input = request.args.get('q')
    # 유사도 계산 및 가장 유사한 텍스트 찾기 (여기서는 로직 구현 필요)
    results = search(user_input)
    return jsonify(results)

if __name__ == '__main__':
    app.run(debug=True)